In [2]:
from autoencoder import AutoEncoder
import pandas as pd
import numpy as np

In [41]:
from torch.utils.data import DataLoader, TensorDataset
import torch

In [42]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
sample = pd.read_csv('predictions.csv')

In [8]:
res = { k: v.get(["book_id", "rating"]) for k, v in sample.groupby('user_id')}

In [34]:
sample_matrix = pd.DataFrame(index= res.keys(), columns = sample['book_id'].unique())


In [35]:
for k, v in res.items():
	t = v.reset_index(drop=True).transpose()
	sample_matrix.loc[k][t.loc["book_id"]] = t.loc["rating"]

In [36]:
sample.shape

(81207, 4)

In [37]:
sample_matrix.notna().sum().sum()

80750

In [39]:
sample_matrix = sample_matrix.fillna(0)

In [45]:
sample_matrix.to_csv('sample_matrix.csv.gzip', index=True, columns=sample_matrix.columns, chunksize=1000, compression='gzip')

In [ ]:
bottleneck = 32

In [ ]:
tensors = torch.tensor(sample_matrix.T.to_numpy(), dtype=torch.float32, device=device)
batch_size = 32 
train = TensorDataset(tensors)
batches = DataLoader(train, batch_size=batch_size, shuffle=True)
model = AutoEncoder(sample_matrix.shape[1], bottleneck)
epochs, losses = model.fit(batches, n_epochs=1000, lr=0.001, patience=10)
pd.DataFrame({"epochs": epochs, "training losses": losses}).to_csv(f'models/training_loss/{bottleneck}.csv')

torch.save(model, f'sample_model_k={bottleneck}.pt')
torch.save(model.state_dict(), f'sample_model_k_weights_{bottleneck}.pt')